In [1]:
import argparse
import importlib
import multiprocessing as mp
import os
import pickle
import yaml

import node2vec as n2v
import numpy as np
from sklearn import decomposition as sk_dc
from sklearn import pipeline as sk_pipe
from sklearn import preprocessing as sk_prep
from tqdm.auto import tqdm

import sys; sys.path.append('../../')
from attre2vec import utils as ae_utils

In [2]:
def save_pkl(path, obj):
    with open(path, 'wb') as fout:
        pickle.dump(obj=obj, file=fout)

###  Hyperparameters

In [104]:
hps = {
    'methods': ['dw', 'n2v', 'sdne', 'struc2vec'],
    'dims': [8, 16, 32, 64, 128],
    'ops': ['avg', 'hadamard', 'l1', 'l2'],
}

## Datasets

In [4]:
with open('../../data/datasets/cora.pkl', 'rb') as fin:
    ds = pickle.load(fin)

In [8]:
!mkdir -p bdata/
!ls

baselines-check.ipynb  bdata  hidecode.tpl  summarize-metrics.ipynb


In [9]:
node_embs = {
    method: {}
    for method in hps['methods']
}

### Deep Walk + Node2vec

In [10]:
def run_node2vec(graph, p, q, dim):
    """Obtains node embeddings using Node2vec."""
    emb = n2v.Node2Vec(
        graph=graph,
        dimensions=dim,
        workers=mp.cpu_count(),
        p=p,
        q=q,
        quiet=True,
    ).fit()

    emb = {
        node_id: emb.wv[str(node_id)]
        for node_id in sorted(graph.nodes())
    }
    return emb

In [11]:
# DW
os.makedirs('bdata/embs/dw', exist_ok=True)
for dim in tqdm(hps['dims'], desc='DW-dims'):
    emb_dw = run_node2vec(graph=ds['original_graph'], p=1, q=1, dim=dim)
    save_pkl(f'bdata/embs/dw/{dim}.pkl', emb_dw)
    node_embs['dw'][dim] = emb_dw

In [12]:
# Node2vec
os.makedirs('bdata/embs/n2v', exist_ok=True)
for dim in tqdm(hps['dims'], desc='N2V-dims'):
    emb_n2v = run_node2vec(graph=ds['original_graph'], p=4, q=1, dim=dim)
    save_pkl(f'bdata/embs/n2v/{dim}.pkl', emb_n2v)
    node_embs['n2v'][dim] = emb_n2v

## SDNE

In [60]:
from gem.embedding.sdne import SDNE

def run_sdne(graph, dim):
    mod = SDNE(
        d=dim,
        beta=5, alpha=1e-5,
        nu1=1e-6, nu2=1e-6, 
        K=3, n_units=[50, 15,],
        n_iter=50, xeta=0.01, 
        n_batch=500,
    )
    mod.learn_embedding(graph=graph, is_weighted=False)
    np_emb = mod.get_embedding()
    
    emb = {
        node_id: np_emb[node_id]
        for node_id in sorted(graph.nodes())
    }
    return emb

In [63]:
os.makedirs('bdata/embs/sdne', exist_ok=True)
for dim in tqdm(hps['dims'], desc='SDNE-dims'):
    emb_sdne = run_sdne(graph=ds['original_graph'], dim=dim)
    save_pkl(f'bdata/embs/sdne/{dim}.pkl', emb_sdne)
    node_embs['sdne'][dim] = emb_sdne

/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(15, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:73: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[K - 1])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  encoder = Model(input=x, output=y[K])
/h

Epoch 1/50
20/20 [==============================] - 1s - loss: 12.0545 - merge_25_loss: 6.0092 - merge_26_loss: 6.0123 - merge_27_loss: 0.0015     
Epoch 2/50
20/20 [==============================] - 0s - loss: 11.8538 - merge_25_loss: 5.9086 - merge_26_loss: 5.9123 - merge_27_loss: 0.0021     
Epoch 3/50
20/20 [==============================] - 0s - loss: 11.7205 - merge_25_loss: 5.8435 - merge_26_loss: 5.8441 - merge_27_loss: 0.0029     
Epoch 4/50
20/20 [==============================] - 0s - loss: 11.6757 - merge_25_loss: 5.8187 - merge_26_loss: 5.8242 - merge_27_loss: 0.0040     
Epoch 5/50
20/20 [==============================] - 0s - loss: 11.6603 - merge_25_loss: 5.8127 - merge_26_loss: 5.8148 - merge_27_loss: 0.0053     
Epoch 6/50
20/20 [==============================] - 0s - loss: 11.6293 - merge_25_loss: 5.7980 - merge_26_loss: 5.7985 - merge_27_loss: 0.0068     
Epoch 7/50
20/20 [==============================] - 0s - loss: 11.6147 - merge_25_loss: 5.7913 - merge_26_loss: 

/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(15, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:73: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[K - 1])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  encoder = Model(input=x, output=y[K])
/

Epoch 1/50
20/20 [==============================] - 1s - loss: 12.0463 - merge_28_loss: 6.0086 - merge_29_loss: 6.0044 - merge_30_loss: 0.0053     
Epoch 2/50
20/20 [==============================] - 0s - loss: 11.8563 - merge_28_loss: 5.9125 - merge_29_loss: 5.9107 - merge_30_loss: 0.0076     
Epoch 3/50
20/20 [==============================] - 0s - loss: 11.7227 - merge_28_loss: 5.8455 - merge_29_loss: 5.8441 - merge_30_loss: 0.0134     
Epoch 4/50
20/20 [==============================] - 0s - loss: 11.6725 - merge_28_loss: 5.8182 - merge_29_loss: 5.8212 - merge_30_loss: 0.0261     
Epoch 5/50
20/20 [==============================] - 0s - loss: 11.6597 - merge_28_loss: 5.8132 - merge_29_loss: 5.8134 - merge_30_loss: 0.0532     
Epoch 6/50
20/20 [==============================] - 0s - loss: 11.6105 - merge_28_loss: 5.7891 - merge_29_loss: 5.7883 - merge_30_loss: 0.1437     
Epoch 7/50
20/20 [==============================] - 0s - loss: 11.5249 - merge_28_loss: 5.7467 - merge_29_loss: 

/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(15, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:73: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[K - 1])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  encoder = Model(input=x, output=y[K])
/

Epoch 1/50
20/20 [==============================] - 1s - loss: 12.0658 - merge_31_loss: 6.0145 - merge_32_loss: 6.0177 - merge_33_loss: 0.0070     
Epoch 2/50
20/20 [==============================] - 0s - loss: 11.8564 - merge_31_loss: 5.9102 - merge_32_loss: 5.9126 - merge_33_loss: 0.0099     
Epoch 3/50
20/20 [==============================] - 0s - loss: 11.7273 - merge_31_loss: 5.8484 - merge_32_loss: 5.8454 - merge_33_loss: 0.0152     
Epoch 4/50
20/20 [==============================] - 0s - loss: 11.6775 - merge_31_loss: 5.8235 - merge_32_loss: 5.8206 - merge_33_loss: 0.0227     
Epoch 5/50
20/20 [==============================] - 0s - loss: 11.6573 - merge_31_loss: 5.8118 - merge_32_loss: 5.8121 - merge_33_loss: 0.0363     
Epoch 6/50
20/20 [==============================] - 0s - loss: 11.6263 - merge_31_loss: 5.7968 - merge_32_loss: 5.7961 - merge_33_loss: 0.0694     
Epoch 7/50
20/20 [==============================] - 0s - loss: 11.5503 - merge_31_loss: 5.7584 - merge_32_loss: 

/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(15, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:73: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[K - 1])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  encoder = Model(input=x, output=y[K])
/

Epoch 1/50
20/20 [==============================] - 2s - loss: 12.0907 - merge_34_loss: 6.0286 - merge_35_loss: 6.0280 - merge_36_loss: 0.0059     
Epoch 2/50
20/20 [==============================] - 1s - loss: 11.9049 - merge_34_loss: 5.9355 - merge_35_loss: 5.9354 - merge_36_loss: 0.0158     
Epoch 3/50
20/20 [==============================] - 1s - loss: 11.7771 - merge_34_loss: 5.8710 - merge_35_loss: 5.8722 - merge_36_loss: 0.0394     
Epoch 4/50
20/20 [==============================] - 0s - loss: 11.7267 - merge_34_loss: 5.8482 - merge_35_loss: 5.8445 - merge_36_loss: 0.0828     
Epoch 5/50
20/20 [==============================] - 0s - loss: 11.7053 - merge_34_loss: 5.8363 - merge_35_loss: 5.8350 - merge_36_loss: 0.1582     
Epoch 6/50
20/20 [==============================] - 0s - loss: 11.6248 - merge_34_loss: 5.7959 - merge_35_loss: 5.7950 - merge_36_loss: 0.4591     
Epoch 7/50
20/20 [==============================] - 0s - loss: 11.5377 - merge_34_loss: 5.7521 - merge_35_loss: 

/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:71: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(15, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[i])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:73: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_regularizer=<keras.reg...)`
  W_regularizer=Reg.l1_l2(l1=nu1, l2=nu2))(y[K - 1])
/home/peter/venvs/egs/lib/python3.7/site-packages/gem/embedding/sdne_utils.py:75: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  encoder = Model(input=x, output=y[K])


Epoch 1/50
20/20 [==============================] - 2s - loss: 12.0892 - merge_37_loss: 6.0275 - merge_38_loss: 6.0269 - merge_39_loss: 0.0093     
Epoch 2/50
20/20 [==============================] - 0s - loss: 11.9113 - merge_37_loss: 5.9363 - merge_38_loss: 5.9403 - merge_39_loss: 0.0068     
Epoch 3/50
20/20 [==============================] - 0s - loss: 11.7723 - merge_37_loss: 5.8694 - merge_38_loss: 5.8682 - merge_39_loss: 0.0089     
Epoch 4/50
20/20 [==============================] - 0s - loss: 11.7174 - merge_37_loss: 5.8418 - merge_38_loss: 5.8409 - merge_39_loss: 0.0114     
Epoch 5/50
20/20 [==============================] - 0s - loss: 11.6743 - merge_37_loss: 5.8209 - merge_38_loss: 5.8188 - merge_39_loss: 0.0140     
Epoch 6/50
20/20 [==============================] - 0s - loss: 11.6411 - merge_37_loss: 5.8035 - merge_38_loss: 5.8031 - merge_39_loss: 0.0162     
Epoch 7/50
20/20 [==============================] - 0s - loss: 11.6292 - merge_37_loss: 5.7971 - merge_38_loss: 

### Struc2vec

In [79]:
!pip install git+https://github.com/shenweichen/GraphEmbedding.git

in _manylinux.py
  Cloning https://github.com/shenweichen/GraphEmbedding.git to /tmp/pip-req-build-1wbmr7dc
    100% |████████████████████████████████| 23.2MB 866kB/s ta 0:00:01    54% |█████████████████▌              | 12.7MB 1.2MB/s eta 0:00:10
    100% |████████████████████████████████| 1.6MB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 276kB 1.4MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 2.0MB/s ta 0:00:01
  Stored in directory: /tmp/pip-ephem-wheel-cache-ak3ypks4/wheels/f0/22/a6/74f1fd14f89481a018365de75b181dfb98ac3c1f42cd3e3b28
  Stored in directory: /home/peter/.cache/pip/wheels/97/37/bd/2b80cbc3be93cdf9dd3348dc8b5e659e059f5b51fcecc706be
  Stored in directory: /home/peter/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
  Stored in directory: /home/peter/.cache/pip/wheels/61/62/46/5a9cc316aa50cc4fb36fd4bf14e7dd206642db3247d7746c23
Successfully built ge gensim networkx fastdtw
node2vec 0.3.1 has requirement jobl

In [103]:
import networkx as nx
from ge import Struc2Vec


def run_struc2vec(graph, dim):
    g = nx.relabel_nodes(G=graph, mapping={n: str(n) for n in graph.nodes()})
    model = Struc2Vec(g, workers=1, verbose=0)
    model.train(embed_size=dim, window_size=5, iter=3)
    e = model.get_embeddings()
        
    emb = {
        node_id: e[str(node_id)]
        for node_id in sorted(graph.nodes())
    }
    return emb

In [106]:
os.makedirs('bdata/embs/struc2vec', exist_ok=True)
for dim in tqdm(hps['dims'], desc='Struc2vec-dims'):
    emb_s2v = run_struc2vec(graph=ds['original_graph'], dim=dim)
    save_pkl(f'bdata/embs/struc2vec/{dim}.pkl', emb_s2v)
    node_embs['struc2vec'][dim] = emb_s2v

Learning representation...
Learning representation done!
Learning representation...
Learning representation done!
Learning representation...
Learning representation done!
Learning representation...
Learning representation done!
Learning representation...
Learning representation done!



### To edge embedding methods

In [107]:
def to_edge_emb(name, edges, node_emb):
    e_emb = {}
    
    for u, v in edges:
        if name == 'avg':
            e_emb[(u, v)] = np.mean([node_emb[u], node_emb[v]], axis=0)
        elif name == 'hadamard':
            e_emb[(u, v)] = node_emb[u] * node_emb[v]
        elif name == 'l1':
            e_emb[(u, v)] = np.abs(node_emb[u] - node_emb[v])
        elif name == 'l2':
            e_emb[(u, v)] = np.power(node_emb[u] - node_emb[v], 2)
    return e_emb
            
            
            
all_edges = list(ds['original_graph'].edges())
edge_emb = {
    method: {
        d: {
            op: to_edge_emb(op, all_edges, node_embs[method][d])
            for op in hps['ops']
        }
        for d in hps['dims']
    }
    for method in hps['methods']
}

### Evaluate

In [117]:
from sklearn import linear_model as lm

def eval_clf(e_emb):
    accs = {tt: [] for tt in ('train', 'val', 'test')}
    aucs = {tt: [] for tt in ('train', 'val', 'test')}
    f1s = {tt: [] for tt in ('train', 'val', 'test')}

    for idx in tqdm(range(ds['num_datasets']), desc='Datasets', leave=False):
        x = {
            tt: np.array([e_emb[e] for e in ds['Xy'][idx][tt]['X']])
            for tt in ('train', 'val', 'test')
        }
        y = {tt: ds['Xy'][idx][tt]['y'] for tt in ('train', 'val', 'test')}

        # Train
        model = lm.LogisticRegression(max_iter=500, multi_class='multinomial')
        model.fit(x['train'], y['train'])

        # Eval
        for tt in ('train', 'val', 'test'):
            m = ae_utils.calc_metrics(
                y_score=model.predict_proba(x[tt]),
                y_pred=model.predict(x[tt]),
                y_true=y[tt],
                max_cls=ds['num_cls'],
            )
            accs[tt].append(m['accuracy'] * 100.)
            aucs[tt].append(m['auc'] * 100.)
            f1s[tt].append(m['macro avg']['f1-score'] * 100.)
    
    accs = {k: f'{np.mean(v):.2f} +/- {np.std(v):.2f}' for k, v in accs.items()}
    aucs = {k: f'{np.mean(v):.2f} +/- {np.std(v):.2f}' for k, v in aucs.items()}
    f1s = {k: f'{np.mean(v):.2f} +/- {np.std(v):.2f}' for k, v in f1s.items()}
    
    return (
        accs['train'], accs['val'], accs['test'],
        aucs['train'], aucs['val'], aucs['test'],
        f1s['train'], f1s['val'], f1s['test'],
    )


records = []
for method in tqdm(hps['methods'], desc='Method'):
    for d in hps['dims']:
        for op in hps['ops']:
            records.append((method, d, op, *eval_clf(edge_emb[method][d][op])))

In [120]:
import pandas as pd

df = pd.DataFrame.from_records(records, columns=[
    'method', 'dim', 'op', 
    'acc_train', 'acc_val', 'acc_test',
    'auc_train', 'auc_val', 'auc_test',
    'f1_train', 'f1_val', 'f1_test',
])

df.head()

,method,dim,op,acc_train,acc_val,acc_test,auc_train,auc_val,auc_test,f1_train,f1_val,f1_test
0,dw,8,avg,29.25 +/- 2.40,16.40 +/- 1.76,15.97 +/- 1.65,69.65 +/- 1.29,55.12 +/- 1.52,54.69 +/- 0.86,26.96 +/- 2.85,14.69 +/- 1.69,14.30 +/- 1.47
1,dw,8,hadamard,26.94 +/- 2.61,12.40 +/- 0.52,12.83 +/- 1.39,67.48 +/- 2.26,50.35 +/- 1.30,51.09 +/- 1.39,25.64 +/- 2.69,11.07 +/- 0.56,11.73 +/- 1.35
2,dw,8,l1,27.44 +/- 2.55,13.17 +/- 1.71,13.09 +/- 1.87,68.99 +/- 1.24,51.38 +/- 1.22,52.00 +/- 1.85,25.68 +/- 2.91,11.95 +/- 1.20,11.98 +/- 1.34
3,dw,8,l2,24.56 +/- 3.09,14.09 +/- 2.18,13.40 +/- 1.83,66.80 +/- 2.08,51.84 +/- 0.96,52.15 +/- 1.36,22.40 +/- 3.57,12.15 +/- 1.20,11.49 +/- 1.03
4,dw,16,avg,37.62 +/- 3.61,17.11 +/- 1.40,16.60 +/- 1.70,76.33 +/- 1.17,56.34 +/- 1.92,55.85 +/- 0.91,36.12 +/- 3.59,16.15 +/- 1.18,15.57 +/- 1.51


### Deep Walk results

In [69]:
df_dw = df[df.method == 'dw'][['method', 'dim', 'op', 'auc_test', 'acc_test', 'f1_test']]

df_dw

,method,dim,op,auc_test,acc_test,f1_test
0,dw,8,avg,54.69 +/- 0.86,15.97 +/- 1.65,14.30 +/- 1.47
1,dw,8,hadamard,51.09 +/- 1.39,12.83 +/- 1.39,11.73 +/- 1.35
2,dw,8,l1,52.00 +/- 1.85,13.09 +/- 1.87,11.98 +/- 1.34
3,dw,8,l2,52.15 +/- 1.36,13.40 +/- 1.83,11.49 +/- 1.03
4,dw,16,avg,55.85 +/- 0.91,16.60 +/- 1.70,15.57 +/- 1.51
5,dw,16,hadamard,50.95 +/- 1.40,13.13 +/- 1.34,11.69 +/- 1.28
6,dw,16,l1,51.98 +/- 1.65,13.04 +/- 1.82,11.99 +/- 1.42
7,dw,16,l2,52.24 +/- 1.16,13.55 +/- 1.82,12.31 +/- 1.10
8,dw,32,avg,59.07 +/- 1.47,20.71 +/- 1.66,19.72 +/- 1.41
9,dw,32,hadamard,51.84 +/- 1.23,13.03 +/- 1.23,11.77 +/- 0.93


In [70]:
df_dw[df_dw.op == 'avg']

,method,dim,op,auc_test,acc_test,f1_test
0,dw,8,avg,54.69 +/- 0.86,15.97 +/- 1.65,14.30 +/- 1.47
4,dw,16,avg,55.85 +/- 0.91,16.60 +/- 1.70,15.57 +/- 1.51
8,dw,32,avg,59.07 +/- 1.47,20.71 +/- 1.66,19.72 +/- 1.41
12,dw,64,avg,60.14 +/- 1.30,22.73 +/- 1.14,22.09 +/- 1.09
16,dw,128,avg,62.14 +/- 1.43,26.31 +/- 1.05,25.36 +/- 0.92


### Node2vec results

In [71]:
df_n2v = df[df.method == 'n2v'][['method', 'dim', 'op', 'auc_test', 'acc_test', 'f1_test']]

df_n2v

,method,dim,op,auc_test,acc_test,f1_test
20,n2v,8,avg,54.90 +/- 0.85,16.31 +/- 1.36,14.56 +/- 1.21
21,n2v,8,hadamard,51.14 +/- 1.43,13.02 +/- 1.47,11.70 +/- 1.30
22,n2v,8,l1,51.91 +/- 1.89,13.09 +/- 1.84,12.05 +/- 1.36
23,n2v,8,l2,52.14 +/- 1.40,13.63 +/- 2.11,11.98 +/- 1.48
24,n2v,16,avg,55.99 +/- 1.00,16.63 +/- 1.47,15.69 +/- 1.28
25,n2v,16,hadamard,50.90 +/- 1.36,13.08 +/- 1.32,11.55 +/- 1.04
26,n2v,16,l1,52.06 +/- 1.58,13.30 +/- 1.47,12.36 +/- 1.18
27,n2v,16,l2,52.40 +/- 1.07,13.41 +/- 1.60,12.34 +/- 1.33
28,n2v,32,avg,59.19 +/- 1.48,21.02 +/- 1.79,19.95 +/- 1.46
29,n2v,32,hadamard,51.89 +/- 1.12,13.43 +/- 1.00,11.73 +/- 0.70


In [72]:
df_n2v[df_n2v.op == 'avg']

,method,dim,op,auc_test,acc_test,f1_test
20,n2v,8,avg,54.90 +/- 0.85,16.31 +/- 1.36,14.56 +/- 1.21
24,n2v,16,avg,55.99 +/- 1.00,16.63 +/- 1.47,15.69 +/- 1.28
28,n2v,32,avg,59.19 +/- 1.48,21.02 +/- 1.79,19.95 +/- 1.46
32,n2v,64,avg,60.28 +/- 1.21,23.24 +/- 1.34,22.50 +/- 1.20
36,n2v,128,avg,62.48 +/- 1.47,26.72 +/- 1.25,25.87 +/- 1.25


### SDNE results

In [73]:
df_sdne = df[df.method == 'sdne'][['method', 'dim', 'op', 'auc_test', 'acc_test', 'f1_test']]

df_sdne

,method,dim,op,auc_test,acc_test,f1_test
40,sdne,8,avg,55.35 +/- 2.27,15.66 +/- 2.78,13.06 +/- 2.11
41,sdne,8,hadamard,52.95 +/- 2.62,13.47 +/- 1.91,10.13 +/- 1.40
42,sdne,8,l1,52.22 +/- 1.53,14.00 +/- 1.42,11.80 +/- 1.09
43,sdne,8,l2,52.44 +/- 1.49,14.85 +/- 2.77,11.63 +/- 1.42
44,sdne,16,avg,53.75 +/- 2.19,15.24 +/- 2.00,13.90 +/- 1.72
45,sdne,16,hadamard,52.75 +/- 1.62,15.44 +/- 1.72,13.12 +/- 1.42
46,sdne,16,l1,51.69 +/- 1.05,13.94 +/- 0.96,12.78 +/- 0.87
47,sdne,16,l2,51.59 +/- 0.69,13.41 +/- 0.88,12.01 +/- 0.47
48,sdne,32,avg,54.53 +/- 1.80,15.77 +/- 2.11,14.30 +/- 1.78
49,sdne,32,hadamard,52.59 +/- 1.65,14.43 +/- 1.62,13.03 +/- 1.51


In [75]:
df_sdne[df_sdne.op == 'avg']

,method,dim,op,auc_test,acc_test,f1_test
40,sdne,8,avg,55.35 +/- 2.27,15.66 +/- 2.78,13.06 +/- 2.11
44,sdne,16,avg,53.75 +/- 2.19,15.24 +/- 2.00,13.90 +/- 1.72
48,sdne,32,avg,54.53 +/- 1.80,15.77 +/- 2.11,14.30 +/- 1.78
52,sdne,64,avg,54.98 +/- 1.46,15.45 +/- 1.26,14.24 +/- 0.99
56,sdne,128,avg,54.21 +/- 1.11,15.47 +/- 2.83,13.72 +/- 2.25


### Struc2vec

In [121]:
df_s2v = df[df.method == 'struc2vec'][['method', 'dim', 'op', 'auc_test', 'acc_test', 'f1_test']]

df_s2v

,method,dim,op,auc_test,acc_test,f1_test
60,struc2vec,8,avg,53.24 +/- 1.53,15.29 +/- 1.67,14.48 +/- 1.60
61,struc2vec,8,hadamard,51.17 +/- 0.97,13.49 +/- 1.39,11.98 +/- 0.93
62,struc2vec,8,l1,54.06 +/- 1.29,15.43 +/- 1.80,14.30 +/- 1.75
63,struc2vec,8,l2,54.44 +/- 1.14,15.51 +/- 1.20,14.51 +/- 1.24
64,struc2vec,16,avg,57.05 +/- 1.29,18.37 +/- 1.34,17.73 +/- 0.98
65,struc2vec,16,hadamard,53.45 +/- 1.12,14.98 +/- 1.09,14.30 +/- 1.16
66,struc2vec,16,l1,55.65 +/- 1.66,16.69 +/- 1.40,15.96 +/- 1.52
67,struc2vec,16,l2,55.70 +/- 1.31,16.25 +/- 1.29,15.69 +/- 1.19
68,struc2vec,32,avg,58.70 +/- 1.53,20.28 +/- 1.57,19.76 +/- 1.43
69,struc2vec,32,hadamard,53.90 +/- 1.34,15.91 +/- 1.89,14.97 +/- 1.66


In [122]:
df_s2v[df_s2v.op == 'avg']

,method,dim,op,auc_test,acc_test,f1_test
60,struc2vec,8,avg,53.24 +/- 1.53,15.29 +/- 1.67,14.48 +/- 1.60
64,struc2vec,16,avg,57.05 +/- 1.29,18.37 +/- 1.34,17.73 +/- 0.98
68,struc2vec,32,avg,58.70 +/- 1.53,20.28 +/- 1.57,19.76 +/- 1.43
72,struc2vec,64,avg,61.80 +/- 1.64,22.87 +/- 2.15,22.42 +/- 1.77
76,struc2vec,128,avg,61.16 +/- 1.71,22.26 +/- 1.96,21.71 +/- 1.65
